In [6]:
from tensorzero import AsyncTensorZero
import random

In [7]:
tensorzero_client = AsyncTensorZero("http://localhost:3000")

In [8]:
with open("nounlist.txt", "r") as file:
    nouns = [line.strip() for line in file]
    random.shuffle(nouns)


In [25]:
async def write_grade_haiku(topic: str, client: AsyncTensorZero) -> (str, bool):
    haiku_result = await client.inference(
        function_name="write_haiku",
        input={
            "messages": [
                {
                    "role": "user",
                    "content": [{"type": "text", "value": {"topic": topic}}],
                }
            ]
        },
    )
    print("here")
    haiku_text = haiku_result.output[0].text
    haiku_lines = haiku_text.split("\n")
    haiku_text = "\n".join(haiku_lines[-3:])
    judge_result = await client.inference(
        function_name="judge_haiku",
        input={
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "value": {"topic": topic, "haiku": haiku_text}}
                    ],
                }
            ]
        },
    )
    score = judge_result.output.parsed["score"]
    # reasoning = judge_result.output.parsed["reasoning"]
    return haiku_text, score

In [26]:
for noun in nouns:
    haiku, score = await write_grade_haiku(noun, tensorzero_client)
    print(f"{noun}: \n{haiku}\n Passed: {score}")
    break

ReadTimeout: 